# House price predection using regression model

# 1.0 Import and install python libraries

In [38]:
#Here we import any Python libraries that we plan to use
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor 
from sklearn.dummy import DummyRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

# 2.0 Set Random Seed

In [39]:
# set random seed to ensure that results are repeatable
np.random.seed(1)

# 3.0 Load data

In [40]:
# load data
housedata = pd.read_csv(r"C:\DSP\Assignment 1\kc_house_data.csv")

# 4.0 Conduct initial exploration of the data

In [41]:
# look at the data
housedata.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


### Determine the input and target variables. The  "price" for the house serves as the dataset's target variable. We are attempting to predict this variable using the other variables in the dataset.

The input variables, also known as predictor variables or features, are all the other variables in the dataset that can be used to predict the sale price of the house. These include the number of bedrooms and bathrooms, the square footage of the living area, the year the house was built, and the location of the house (latitude and longitude), among others.

In [42]:
# generate a basic summary of the data
housedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [43]:
# Check the shape of the dataset
housedata.shape

(21613, 21)

In [44]:
# Check the data types of each column
housedata.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [45]:
# generate a statistical summary of the numeric value in the data
housedata.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [46]:
# Check the missing values by summing the total na's for each variable
housedata.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

### Categorical variables are frequently used in regression analysis as explanatory or predictor variables in order to explain the variance in the response variable. In regression models, dummy variables are frequently used to represent categorical variables. To denote the existence or absence of a specific category, a dummy variable has the value of 1 or 0.

In [47]:
# encode categorical variables
cat_cols = ["waterfront", "view", "condition", "grade"]
for col in cat_cols:
    housedata[col] = LabelEncoder().fit_transform(housedata[col])

# 5.0 Preprocess the dataset

### In order to enhance the model's performance, decrease overfitting, and improve interpretability, it is essential to drop predictor variables from a regression model.

In [48]:
#Drop id and Zipcode columns
housedata.drop(['id','date'], axis=1, inplace = True)

In [49]:
housedata = housedata.dropna()

In [50]:
housedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  int64  
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  int64  
 4   sqft_lot       21613 non-null  int64  
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  int64  
 7   view           21613 non-null  int64  
 8   condition      21613 non-null  int64  
 9   grade          21613 non-null  int64  
 10  sqft_above     21613 non-null  int64  
 11  sqft_basement  21613 non-null  int64  
 12  yr_built       21613 non-null  int64  
 13  yr_renovated   21613 non-null  int64  
 14  zipcode        21613 non-null  int64  
 15  lat            21613 non-null  float64
 16  long           21613 non-null  float64
 17  sqft_living15  21613 non-null  int64  
 18  sqft_l

In [51]:
# scale numeric variables
#num_cols = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors"]
#scaler = StandardScaler()
#housedata[num_cols] = scaler.fit_transform(housedata[num_cols])

6.0 Split and Save the data

In [52]:
# split the data into training and testing sets
target = 'price'
X = housedata.drop(target, axis=1)
y = housedata[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
#standardizing the numerical variables
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
# convert the numpy array to a pandas DataFrame

X_train = pd.DataFrame(X_train)

y_train = pd.DataFrame(y_train)

X_test = pd.DataFrame(X_test)

y_test = pd.DataFrame(y_test)

In [55]:
# save the DataFrame to a CSV file

X_train.to_csv(r'C:\DSP\Assignment 1\kc_house_data_X_train.csv', index=False)

y_train.to_csv(r'C:\DSP\Assignment 1\kc_house_data_y_train.csv', index=False)

X_test.to_csv(r'C:\DSP\Assignment 1\kc_house_data_X_test.csv', index=False)

y_test.to_csv(r'C:\DSP\Assignment 1\kc_house_data_y_test.csv', index=False)

In [56]:
# set random seed to ensure that results are repeatable
np.random.seed(1)

In [57]:
#load the cleaned data

X_train = pd.read_csv(r"C:\DSP\Assignment 1\kc_house_data_X_train.csv")

X_test = pd.read_csv(r"C:\DSP\Assignment 1\kc_house_data_X_test.csv")

y_train = pd.read_csv(r"C:\DSP\Assignment 1\kc_house_data_y_train.csv")

y_test = pd.read_csv(r"C:\DSP\Assignment 1\kc_house_data_y_test.csv")

## Fitting Models the data

As we selected price as the target variable, regression modeling would provide the best results and house price prediction. Because the target variable price includes continuous values, we are unable to choose classification models. Instead, depending on the data set we have chosen, we select MSC, RMSE, and R-Squared as performance measures. This dataset includes details on King County home sales, including attributes like square footage, location, and the number of bedrooms and bathrooms. The top performance measures that may be utilized to develop a model to forecast home prices using this dataset are MSC, RMSE, and R-Squared.

## 7.0 MLPRegressor

7.1 Fit and test a MLP Regressor Using Stochastic Gradient Descent

In [61]:
mlp_sgd = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu', solver='sgd', max_iter=500)
mlp_sgd.fit(X_train, y_train)
y_pred = mlp_sgd.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 151314499700.7679
RMSE: 388991.64476986893
R-squared: -0.0009114329968840096


7.2 Fit and test a MLP Regressor Using L2 Regularization

In [59]:
mlp_l2 = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu', solver='adam', alpha=0.0001, max_iter=500)
mlp_l2.fit(X_train, y_train)
y_pred = mlp_l2.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 33549681372.96906
RMSE: 183165.72106420202
R-squared: 0.7780763923747295


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


7.3 Fit and test a MLP Regressor  Using L1 Regularization

In [108]:
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', alpha=0.1, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MSE: 36471534307.191025
RMSE: 190975.2190918786
R-squared: 0.758748991410634


7.4 Fit and test a MLP Regressor  Using ElasticNet Regularization

In [113]:
mlp = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', alpha=0.1, solver='adam', max_iter=200, random_state=42, 
                   early_stopping=True, validation_fraction=0.2, n_iter_no_change=5, tol=1e-4)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnin

MSE: 38499012495.87978
RMSE: 196211.65229384258
R-squared: 0.7453376785277082


7.5 Fit and test a MLP Regressor  Using Early Stopping

In [102]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', early_stopping=True, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnin

MSE: 35967548003.765144
RMSE: 189651.12180993063
R-squared: 0.7620827476215084


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## 8. Keras Sequential model

In [62]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

8.1 Fit and test a Keras Sequential model  Using Stochastic Gradient Descent

In [115]:
# create model stucture
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

model = keras.models.Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

sgd = SGD(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer='sgd')
model.fit(X_train, y_train, epochs=1000, batch_size=32)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)


Epoch 1/1000
541/541 [==============================] - 3s 2ms/step - loss: 10324831737181991562379264.0000
Epoch 2/1000
541/541 [==============================] - 1s 2ms/step - loss: 100831118491648.0000
Epoch 3/1000
541/541 [==============================] - 1s 2ms/step - loss: 130699255808.0000
Epoch 4/1000
541/541 [==============================] - 1s 2ms/step - loss: 130710618112.0000
Epoch 5/1000
541/541 [==============================] - 1s 2ms/step - loss: 130703802368.0000
Epoch 6/1000
541/541 [==============================] - 1s 2ms/step - loss: 130706423808.0000
Epoch 7/1000
541/541 [==============================] - 1s 2ms/step - loss: 130681102336.0000
Epoch 8/1000
541/541 [==============================] - 1s 2ms/step - loss: 130684764160.0000
Epoch 9/1000
541/541 [==============================] - 1s 2ms/step - loss: 130651545600.0000
Epoch 10/1000
541/541 [==============================] - 1s 2ms/step - loss: 130699747328.0000
Epoch 11/1000
541/541 [===================

8.2 Fit and test a Keras Sequential model Using L2 Regularization

In [104]:
# create model stucture
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

model = keras.models.Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, batch_size=32)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/1000
541/541 [==============================] - 4s 5ms/step - loss: 419841900544.0000 - accuracy: 0.0000e+00
Epoch 2/1000
541/541 [==============================] - 3s 6ms/step - loss: 419842195456.0000 - accuracy: 0.0000e+00
Epoch 3/1000
541/541 [==============================] - 3s 6ms/step - loss: 419842162688.0000 - accuracy: 0.0000e+00
Epoch 4/1000
541/541 [==============================] - 3s 5ms/step - loss: 419842031616.0000 - accuracy: 0.0000e+00
Epoch 5/1000
541/541 [==============================] - 2s 3ms/step - loss: 419841966080.0000 - accuracy: 0.0000e+00
Epoch 6/1000
541/541 [==============================] - 3s 6ms/step - loss: 419842293760.0000 - accuracy: 0.0000e+00
Epoch 7/1000
541/541 [==============================] - 2s 3ms/step - loss: 419842064384.0000 - accuracy: 0.0000e+00
Epoch 8/1000
541/541 [==============================] - 3s 5ms/step - loss: 419842260992.0000 - accuracy: 0.0000e+00
Epoch 9/1000
541/541 [==============================] - 4s 6ms/s

8.3 Fit and test a Keras Sequential model Using L1 Regularization

In [105]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1

model = keras.models.Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l1(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, batch_size=32)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/1000
541/541 [==============================] - 3s 2ms/step - loss: 419841900544.0000 - accuracy: 0.0000e+00
Epoch 2/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842195456.0000 - accuracy: 0.0000e+00
Epoch 3/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842162688.0000 - accuracy: 0.0000e+00
Epoch 4/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842031616.0000 - accuracy: 0.0000e+00
Epoch 5/1000
541/541 [==============================] - 1s 2ms/step - loss: 419841966080.0000 - accuracy: 0.0000e+00
Epoch 6/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842293760.0000 - accuracy: 0.0000e+00
Epoch 7/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842064384.0000 - accuracy: 0.0000e+00
Epoch 8/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842260992.0000 - accuracy: 0.0000e+00
Epoch 9/1000
541/541 [==============================] - 1s 2ms/s

8.4 Fit and test a Keras Sequential model  Using ElasticNet Regularization

In [106]:
from keras.models import Sequential
from keras.layers import Dense

model = keras.models.Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, batch_size=32)

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

Epoch 1/1000
541/541 [==============================] - 2s 1ms/step - loss: 419841900544.0000 - accuracy: 0.0000e+00
Epoch 2/1000
541/541 [==============================] - 1s 1ms/step - loss: 419842195456.0000 - accuracy: 0.0000e+00
Epoch 3/1000
541/541 [==============================] - 1s 1ms/step - loss: 419842162688.0000 - accuracy: 0.0000e+00
Epoch 4/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842031616.0000 - accuracy: 0.0000e+00
Epoch 5/1000
541/541 [==============================] - 1s 2ms/step - loss: 419841966080.0000 - accuracy: 0.0000e+00
Epoch 6/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842293760.0000 - accuracy: 0.0000e+00
Epoch 7/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842064384.0000 - accuracy: 0.0000e+00
Epoch 8/1000
541/541 [==============================] - 1s 2ms/step - loss: 419842260992.0000 - accuracy: 0.0000e+00
Epoch 9/1000
541/541 [==============================] - 1s 2ms/s

8.5 Fit and test a Keras Sequential model  Using Early Stopping

In [107]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

model = keras.models.Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')

model.fit(X_train, y_train, validation_data=(X,y), epochs=1000, batch_size=32, callbacks=[early_stop])

# make predictions on the test data
y_pred = model.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)



Epoch 1/1000
541/541 [==============================] - 4s 5ms/step - loss: 419841900544.0000 - accuracy: 0.0000e+00 - val_loss: 426470440960.0000 - val_accuracy: 0.0000e+00
Epoch 2/1000
541/541 [==============================] - 2s 3ms/step - loss: 419842195456.0000 - accuracy: 0.0000e+00 - val_loss: 426470440960.0000 - val_accuracy: 0.0000e+00
Epoch 3/1000
541/541 [==============================] - 2s 3ms/step - loss: 419842162688.0000 - accuracy: 0.0000e+00 - val_loss: 426470440960.0000 - val_accuracy: 0.0000e+00
Epoch 4/1000
541/541 [==============================] - 2s 3ms/step - loss: 419842031616.0000 - accuracy: 0.0000e+00 - val_loss: 426470440960.0000 - val_accuracy: 0.0000e+00
Epoch 5/1000
541/541 [==============================] - 2s 3ms/step - loss: 419841966080.0000 - accuracy: 0.0000e+00 - val_loss: 426470440960.0000 - val_accuracy: 0.0000e+00
Epoch 6/1000
136/136 [==============================] - 0s 1ms/step
MSE: 452980201757.603
RMSE: 673038.03886378
R-squared: -1.9963

## 9. Linear Regression

9.1 Fit and test a Linear Regression Using Stochastic Gradient Descent

In [90]:
# create an instance of the SGDRegressor class
reg = SGDRegressor(max_iter=1000, tol=1e-3, alpha=0.01, random_state=42)

# fit the model on the training data
reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = reg.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 45729427451.15249
RMSE: 213844.40009304075
R-squared: 0.6975101074201431


9.2 Fit and test SGDRegression using L2 Regularization

In [91]:
# create an instance of the SGDRegressor class with L2 regularization
reg = SGDRegressor(max_iter=1000, tol=1e-3, alpha=0.01, penalty='l2', random_state=42)

# fit the model on the training data
reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = reg.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 45729427451.15249
RMSE: 213844.40009304075
R-squared: 0.6975101074201431


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


9.3 Fit and test SGDRegression Using L1 Regularization

In [93]:
# create an instance of the SGDRegressor class with L1 regularization
reg = SGDRegressor(max_iter=1000, tol=1e-3, alpha=0.01, penalty='l1', random_state=42)

# fit the model on the training data
reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = reg.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 45664161353.02602
RMSE: 213691.7437642971
R-squared: 0.6979418280016543


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


9.4 Fit and test SGDRegression using ElasticNet Regularization

In [94]:
# create an instance of the SGDRegressor with Elastic Net regularization:
reg = SGDRegressor(max_iter=1000, tol=1e-3, alpha=0.01, penalty='elasticnet', random_state=42)

# fit the model on the training data
reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = reg.predict(X_test)

# evaluate the model using RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 45719010762.441444
RMSE: 213820.0429390132
R-squared: 0.6975790114765651


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


9.5 Fit and test SGDRegression using Early Stopping

In [95]:
# create an instance of the SGDRegressor with Early stopping:
reg = SGDRegressor(max_iter=1000, tol=1e-3, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, random_state=42)

# fit the model on the training data
reg.fit(X_train, y_train)

# make predictions on the test data
y_pred = reg.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 45326063272.027756
RMSE: 212899.18570071552
R-squared: 0.7001782708766905


## 10. Polynomial Regression

10.1 Fit and test a polynomial Regression Using Stochastic Gradient Descent

In [96]:
#Create a pipeline with PolynomialFeatures and SGDRegressor
pipeline = make_pipeline(PolynomialFeatures(degree=2, include_bias=False), SGDRegressor(random_state=42))

#Fit the model on the training set
pipeline.fit(X_train, y_train)

#Make predictions on the testing set
y_pred = pipeline.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 1.6878132959399367e+25
RMSE: 4108300495265.5747
R-squared: -111645059000373.33


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


10.2 Fit and test a polynomial Regression Using using L2 Regularization

In [97]:
#Create a pipeline with PolynomialFeatures and SGDRegressor
poly = PolynomialFeatures(degree=2, include_bias=False)
sgd_reg = SGDRegressor(penalty='l2', alpha=0.01, random_state=42)
pipeline = make_pipeline(poly, sgd_reg)

#Fit the model on the training set
pipeline.fit(X_train, y_train)

#Make predictions on the testing set
y_pred = pipeline.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 1.5006435785918402e+25
RMSE: 3873814113495.6904
R-squared: -99264202547424.02


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


10.3 Fit and test a polynomial Regression Using using L1 Regularization

In [98]:
poly = PolynomialFeatures(degree=2, include_bias=False)
sgd_reg = SGDRegressor(penalty='l1', alpha=0.01, random_state=42)
pipeline = make_pipeline(poly, sgd_reg)

#Fit the model on the training set
pipeline.fit(X_train, y_train)

#Make predictions on the testing set
y_pred = pipeline.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 1.2529455728353555e+25
RMSE: 3539697123816.3237
R-squared: -82879535751943.39


10.4 Fit and test SGDRegression using ElasticNet Regularization

In [99]:
poly = PolynomialFeatures(degree=2, include_bias=False)
sgd_reg = SGDRegressor(penalty='elasticnet', alpha=0.01, random_state=42)
pipeline = make_pipeline(poly, sgd_reg)

#Fit the model on the training set
pipeline.fit(X_train, y_train)

#Make predictions on the testing set
y_pred = pipeline.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 1.0296670844118014e+24
RMSE: 1014725127515.7235
R-squared: -6811016518616.939


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


10.5 Fit and test SGDRegression using Early Stopping

In [100]:
pipeline = SGDRegressor(alpha=0.01, max_iter=1000, tol=1e-3, early_stopping=True, 
                     validation_fraction=0.2, n_iter_no_change=10, random_state=42)

#Fit the model on the training set
pipeline.fit(X_train, y_train)

#Make predictions on the testing set
y_pred = pipeline.predict(X_test)

# evaluate the model using MSE, RMSE and R-squared metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print('MSE:', mse)
print("RMSE:", rmse)
print("R-squared:", r2)

MSE: 46348214716.046524
RMSE: 215286.3551552827
R-squared: 0.6934169686313945


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


As per the results above the best model is the one with lowest MSE and RMSE value and highest r-squared values. Based on that after using neural network model also we can clearly see that the best performed model is SGDRegression using ElasticNet Regularization model.

Based on the results provided, the best model would be the one with the lowest MSE and RMSE values and the highest R-squared value, as these metrics indicate the accuracy and performance of the model in predicting the target variable.

From the provided results, it seems that the MLPRegressor model with L2 regularization (ridge) has the lowest MSE and RMSE values, and the highest R-squared value among the models listed. Therefore, the MLPRegressor model with L2 regularization (ridge) and early stopping would be considered the best model based on the given metrics.